# <center><font color='OrangeRed'> Анализ базы данных сервиса для чтения книг по подписке</font></center>

Мною будет проведен анализ базы данных сервиса для чтения книг по подписке.

**Задачи**:
- Выяснить, сколько книг вышло после *1 января 2000 года*;
- Для каждой книги посчитать *количество обзоров и среднюю оценку*;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц (так я исключу из анализа брошюры);
- Определите автора с *самой высокой средней оценкой книг* — будут учитываться только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

Данные для анализа:
- Данные о книгах **books**:
    - *book_id* — идентификатор книги;
    - *author_id* — идентификатор автора;
    - *title* — название книги;
    - *num_pages* — количество страниц;
    - *publication_date* — дата публикации книги;
    - *publisher_id* — идентификатор издателя.
- Данные об авторах **authors**:
    - *author_id* — идентификатор автора;
    - *author* — имя автора.
- Данные об издательствах **publishers**:
    - *publisher_id* — идентификатор издательства;
    - *publisher* — название издательства;
- Данные о пользовательских оценках книг **ratings**:
    - *rating_id* — идентификатор оценки;
    - *book_id* — идентификатор книги;
    - *username* — имя пользователя, оставившего оценку;
    - *rating* — оценка книги;
- Данные о пользовательских обзорах на книги **reviews**:
    - *review_id* — идентификатор обзора;
    - *book_id* — идентификатор книги;
    - *username* — имя пользователя, написавшего обзор;
    - *text* — текст обзора.

**Этапы исследования**:
   1. [Изучение и предобработка данных](#obzor)
   2. [Анализ базы данных](#analyz)
   3. [Общий вывод](#vyvod)

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from pandas.io import sql
import sqlite3

# <a id="obzor"></a><font style="background-color: Cyan; color=black">1. Изучение и предобработка данных</font> #

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                        db_config['pwd'],
                                                        db_config['host'],
                                                        db_config['port'],
                                                        db_config['db'])
# Сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Чтобы изучить данные использую формулу, которая по запросу возвращает 2 строки таблицы

In [3]:
def sql(query):
    return pd.io.sql.read_sql(query, con = engine)
def  df_view(table):
    result = sql(f'''
            SELECT
                *
            FROM
                {table}
            LIMIT
                2
            ''')
    return result

In [4]:
def df_viewing(df):
    print(f'Количество дубликатов в таблице - {df.duplicated().sum()}')
    print('Общая информация о датасете') 
    display(df.info()) 

Запрос к базам данных по каждому датафрейму

<font color='green'>Книги</font>

In [5]:
query_1 = ''' SELECT *
            FROM books
        '''
books = pd.io.sql.read_sql(query_1, con = engine) 

In [6]:
df_view('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336


In [7]:
df_viewing(books)

Количество дубликатов в таблице - 0
Общая информация о датасете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

- 6 столбцов; 
- 1 000 строк; 
- Пропуски данных и дубликаты отсутствуют.

<font color='green'>Авторы</font>

In [8]:
query_2 = ''' SELECT *
            FROM authors
        '''
authors = pd.io.sql.read_sql(query_2, con = engine) 

In [9]:
df_view('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs


In [10]:
df_viewing(authors)

Количество дубликатов в таблице - 0
Общая информация о датасете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None

- 2 столбца; 
- 636 строк; 
- Пропуски данных и дубликаты отсутствуют.

<font color='green'>Издательства</font>

In [11]:
query_3 = ''' SELECT *
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query_3, con = engine) 

In [12]:
df_view('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book


In [13]:
df_viewing(publishers)

Количество дубликатов в таблице - 0
Общая информация о датасете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


None

- 2 столбца; 
- 340 строк; 
- Пропуски данных и дубликаты отсутствуют.

<font color='green'>Оценки пользователей</font>

In [14]:
query_4 = ''' SELECT *
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query_4, con = engine) 

In [15]:
df_view('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2


In [16]:
df_viewing(ratings)

Количество дубликатов в таблице - 0
Общая информация о датасете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

- 4 столбца; 
- 6456 строк; 
- Пропуски данных и дубликаты отсутствуют.

<font color='green'>Обзоры на книги</font>

In [17]:
query_5 = ''' SELECT *
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query_5, con = engine) 

In [18]:
df_view('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...


In [19]:
df_viewing(reviews)

Количество дубликатов в таблице - 0
Общая информация о датасете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

- 4 столбца; 
- 2793 строки; 
- Пропуски данных и дубликаты отсутствуют.

Данные корректны, могу приступать к анализу

# <a id="analyz"></a><font style="background-color: Cyan; color=black">2. Анализ базы данных</font> #

<font color='blue'>**Сколько книг вышло после 1 января 2000 года**</font>

In [20]:
query_6 = ''' SELECT
                    COUNT(book_id) AS cnt_book_2000
                FROM
                    books
                WHERE
                   publication_date > '2000-01-01' 
            '''
over_2000 = pd.io.sql.read_sql(query_6, con = engine) 

over_2000

,cnt_book_2000
0,819


После 01.01.2000 выпущено **819 книг**.

<font color='blue'>**Для каждой книги количество обзоров и среднюю оценку**</font>

In [21]:
query_7 = ''' SELECT
                authors.author AS author, 
                books.title, 
                COUNT(DISTINCT reviews.review_id) AS cnt_reviews_book, 
                AVG (ratings.rating) AS median_rating  
            FROM
                authors
                INNER JOIN books ON books.author_id  = authors.author_id
                LEFT JOIN reviews ON books.book_id  = reviews.book_id
                INNER JOIN ratings ON books.book_id  = ratings.book_id
            
            GROUP BY
                  authors.author, 
                books.title 
            ORDER BY 
                authors.author 
            '''
overview = pd.io.sql.read_sql(query_7, con = engine) 

overview

,author,title,cnt_reviews_book,median_rating
0,A.S. Byatt,Possession,3,4.000000
1,Aesop/Laura Harris/Laura Gibbs,Aesop's Fables,2,3.750000
2,Agatha Christie,A Murder Is Announced (Miss Marple #5),2,4.000000
3,Agatha Christie,And Then There Were None,5,4.454545
4,Agatha Christie,Evil Under the Sun (Hercule Poirot #24),2,5.000000
...,...,...,...,...
994,Zadie Smith,On Beauty,2,4.500000
995,Zadie Smith,White Teeth,4,3.750000
996,Zilpha Keatley Snyder,The Egypt Game,2,4.000000
997,Zora Neale Hurston,Their Eyes Were Watching God,4,4.125000


*999 строк* - таблица содержит на <font color='red'>1 книгу меньше, чем по данным о книгах</font>. 

Дополнительно посмотрю на число книг по *ID и по названию*.

In [22]:
query_7_1 = ''' SELECT
                COUNT(DISTINCT books.book_id) 
                
            FROM
                books
            '''
id = pd.io.sql.read_sql(query_7_1, con = engine) 

id

,count
0,1000


In [23]:
query_7_1 = ''' SELECT
                COUNT(DISTINCT books.title) 
                
            FROM
                books
            '''
title = pd.io.sql.read_sql(query_7_1, con = engine) 

title

,count
0,999


В данных присутствует **одна книга с одинаковым названием, но разными ID.**

В итоге получил **999 книг с отзывами (от 2 до 1120 на книгу) и средними оценками от 1,5 до 5**.

<font color='blue'>**Издательство, которое выпустило наибольшее число книг толще 50 страниц (исключаю брошюры)**</font>

In [24]:
query_8 = ''' SELECT
                publishers.publisher AS publishing_house, 
                COUNT(DISTINCT books.book_id) AS cnt_real_book 
            FROM
                publishers
                INNER JOIN books ON books.publisher_id  = publishers.publisher_id
            WHERE
                books.num_pages > 50
            GROUP BY
                  publishers.publisher 
            ORDER BY  
                cnt_real_book DESC
            '''
publishing_house = pd.io.sql.read_sql(query_8, con = engine) 

publishing_house

,publishing_house,cnt_real_book
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19
...,...,...
329,Random House Anchor,1
330,Random House Audio Publishing Group,1
331,Random House Large Print Publishing,1
332,Random House Trade,1


**"Penguin Books"** - издательство, выпустившее наибольшее количество книг (42), толщиной более 50 страниц.

<font color='blue'>**Автор с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками)**</font>

In [25]:
query_9 = ''' SELECT
                authors.author AS author, 
                COUNT(DISTINCT ratings.rating_id) AS cnt_rating_book,
                AVG (ratings.rating) AS median_rating  
            FROM
                authors
                INNER JOIN books ON books.author_id  = authors.author_id
                INNER JOIN ratings ON books.book_id  = ratings.book_id
            GROUP BY
                  authors.author 
            HAVING
                COUNT(DISTINCT ratings.rating_id) > 50
            ORDER BY 
                median_rating DESC
            '''
author_rating = pd.io.sql.read_sql(query_9, con = engine) 

author_rating

,author,cnt_rating_book,median_rating
0,J.K. Rowling/Mary GrandPré,312,4.288462
1,Agatha Christie,53,4.283019
2,Markus Zusak/Cao Xuân Việt Khương,53,4.264151
3,J.R.R. Tolkien,166,4.240964
4,Roald Dahl/Quentin Blake,62,4.209677
5,Louisa May Alcott,54,4.203704
6,Rick Riordan,84,4.130952
7,Arthur Golden,56,4.107143
8,Stephen King,106,4.009434
9,John Grisham,70,3.971429


**Джоан Роулинг совместно с Мэри Гранпрэ** - автор с самым высоким рейтингом, оцененный более, чем 50 пользователями

<font color='blue'>**Среднее количество обзоров от пользователей, которые поставили больше 50 оценок**</font>

In [26]:
query_10 = ''' SELECT
                AVG(cnt_review)  as avg_review
            FROM 
            (SELECT
                reviews.username AS username, 
                COUNT(DISTINCT ratings.rating_id) AS cnt_rating_book,
                COUNT (DISTINCT reviews.review_id) AS cnt_review  
            FROM
                reviews
                INNER JOIN ratings ON ratings.username  = reviews.username
            GROUP BY
                  reviews.username 
            HAVING
                COUNT(DISTINCT ratings.rating_id) > 50
            )
            AS avg_review 
            '''
median_review  = pd.io.sql.read_sql(query_10, con = engine) 

median_review

,avg_review
0,24.333333


**24 обзора** - Среднее количество от пользователей, которые поставили больше 50 оценок.

# <a id="vyvod"></a><font style="background-color: Cyan; color=black">3. Общий вывод</font> #

Мною был проведен анализ баз данных сервиса для чтения книг - *5 таблиц с информацией о книгах, авторах, издательствах, пользовательких оценках и обзорах.*

Был обнаружен **один дубль** - одинаковые названия, разные ID.

По запланированным задачам получены следующие результаты:
- После 01.01.2000 выпущено **819 книг**;
- **999 книг с отзывами (от 2 до 1120 на книгу) и средними оценками от 1,5 до 5**;
- **"Penguin Books"** - издательство, выпустившее наибольшее количество книг (42), толщиной более 50 страниц; на втором месте издательство *Vintage* (31), на третьем - *Grand Central Publishing* (25)
- **Джоан Роулинг совместно с Мэри Гранпрэ** - автор с самым высоким рейтингом (4,289), оцененный более, чем 50 пользователями; Также в топ-5 входят:
    - Агата Кристи - 4,283
    - Маркус Зусак - 4,264
    - Джон Рональд Руэл Толкин - 4,241
    - Роальд Даль - 4,21
- **24 обзора** - Среднее количество от пользователей, которые поставили больше 50 оценок.